# Contextual fraction of 2-qutrit states
## Introduction
This notebook provides an overview of the contextual fraction of 2-qutrit 
states with respect to Heisenberg-Weyl operators.

> This notebook is structured to explain the key concepts and functions used 
> in the project. In each section, the source code of relevant functions is 
> displayed using `inspect.getsource`.

> This notebook will not run as a standalone notebook. You will need to clone the entire repo to run it. 
> Use `src/notebooks/cfraction_code.ipynb` for a standalone notebook with all the functions built in. 

### --- Notebook Structure Summary ---

This notebook is organized into several key sections:

1. Introduction:
   - Provides an overview of the project.
   - Explains the structure of the source code and its pairing with 
     Jupyter notebooks.

2. Heisenberg-Weyl Operators:
   - Introduces the definition of Heisenberg-Weyl operators for 
     2-qutrit states.
   - Displays the source code for the operators module.

3. Qudit Contexts:
   - Describes maximal commuting sets (contexts) of the Heisenberg-Weyl 
     operators.
   - Shows the source code of the contexts module.

4. Projector and Empirical Model:
   - Explains how projectors and empirical models are constructed using 
     spectral decompositions.
   - Prints the source code for the projector and empirical_model functions.

5. Noncontextual and Contextual Fraction:
   - Discusses the concept of global (noncontextual) assignments and the 
     decomposition of empirical models.

6. Optimization Modules:
   - Introduces the incidence matrix and the linear programming formulation 
     used to compute the contextual fraction.
   - Displays the source code for both the incidence_matrix and lin_prog 
     modules.

7. Example Usage:
   - Demonstrates how to compute the contextual fraction for various 
     quantum states.
   - Includes a main function that runs through these computations and 
     prints a summary.

> **Note:** This notebook is paired with a Python script 
> `notebooks/contextual_fraction.py` using 
> [Jupytext](https://jupytext.readthedocs.io/en/latest/). Changes made in 
> either file will be synchronized automatically. The Python script can be 
> run directly as a standalone script.

The source code for this project is structured as follows 
(see `README.md` for more details):
```
2_qudit_contextual_fraction/
|--src/                             # Source code modules
|   |-- utils/                      # Utility modules
|   |   |-- operators.py            # Heisenberg-Weyl operators
|   |   |--contexts.py              # Measurement contexts (40 contexts)
|   |   |--commutators.py           # Commutator checking functions
|   |   |--measurements.py          # Projectors & empirical models
|   |   |--states.py                # Quantum state creation & analysis
|   |   |--ternary.py               # Base-3 number conversion
|   |-- optimization/               # Linear programming optimization
|       |-- incidence_matrix.py     # Global assignment constraint matrix
|       |-- lin_prog.py             # Contextual fraction calculation
|-- notebooks/                      # Jupyter notebooks
|   |-- cfraction_code.ipynb        # Standalone notebook
|   |-- cfraction_code.py           # Jupytext paired Python file
|   |-- contextual_fraction.ipynb   # Main explanation notebook --> Here!
|   |-- contextual_fraction.py      # Jupytext paired Python file
|-- main.py                         # Main execution script
|-- example.py                      # Simple usage examples
|-- README.md                       # Project documentation
```

In [52]:
import sys
import inspect
from IPython.display import Markdown

# Add the parent directory to sys.path so 'src' can be imported
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Also add the src directory directly to the path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

## Heisenberg-Weyl operators
For odd prime dimension $d$, the single qudit Pauli operators are defined by

\begin{equation*}
 X \left|j\right> = \left|j + 1\right> \quad \text{and} \quad 
 Z \left|j\right> = \omega^j \left|j\right>
\end{equation*}

where $\omega = e^{2\pi i/d}$ and the addition is modulo $d$. The 
Heisenberg-Weyl group is generated by the Pauli operators $X$ and $Z$. 

>An element of the 2-qudit Heisenberg-Weyl group is specified by a 
>symplectic vector $v=[p_1,q_1,p_2,q_2] \in \mathbb{Z}_3^4$: 
>\begin{equation*}
> W(v) = W([p_1,q_1,p_2,q_2]) = \omega^{2(p_1q_q+p_2q_2)}X^{p_1} Z^{q_1} 
> \otimes X^{p_2}Z^{q_2}
>\end{equation*}

where we have chosen the phase factor such that

\begin{equation*}
 W(v)W(v') = W(v + v'). 
\end{equation*}

For a given vector $v \in \mathbb{Z}_3^4$, we can generate the corresponding 
Heisenberg-Weyl operator using the function `src/utils.operators.pauli`.

In [53]:
# Printing the source code of the operators module
import utils.operators as operators
Markdown(f"```python\n{inspect.getsource(operators)}\n```")

```python
"""Heisenberg-Weyl Operators generated from symplectic vectors"""
import numpy as np

# Phase factor
w = np.exp(2 * np.pi * 1j / 3)

# Generators of the Heisenberg-Weyl group for qutrits:
X = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])  # Pauli X operator
Z = np.array([[1, 0, 0], [0, w, 0], [0, 0, w**2]])  # Pauli Z operator


# Function to generate the Heisenberg-Weyl operator from symplectic vectors
def pauli(A):
    """
    Generate the Heisenberg-Weyl operator from a symplectic vector.

    Parameters:
        A (iterable of int): A symplectic vector of length 4. 

    Returns:
        numpy.ndarray: The resulting Heisenberg-Weyl operator as a matrix,
            obtained by applying the Kronocker product to the two 
            single-qudit components.
    """
    phase = w**(2 * A[0] * A[1] + 2 * A[2] * A[3])
    first_component = (np.linalg.matrix_power(X, A[0]) @ 
                      np.linalg.matrix_power(Z, A[1]))
    second_component = (np.linalg.matrix_power(X, A[2]) @ 
                       np.linalg.matrix_power(Z, A[3]))
    return phase * np.kron(first_component, second_component)


 

```

## Qudit contexts
A context is a set of pairwise commuting observables. We want to identify 
the maximal contexts of 2-qudit Heisenberg-Weyl operators. 

The operators $W(v)$ and $W(v')$ commute if the vectors $v$ and $v'$ are 
orthogonal with respect to the symplectic inner product:
\begin{equation*}
  [v, v'] = \sum_{i=1}^n (p_i q'_i - p'_i q_i) = 0. 
\end{equation*}

To identify maximal contexts, we need to identify maximal isotropic 
subspaces (i.e, sets of pairwise orthogonal vectors) of the symplectic 
vector space $\mathbb{Z}_3^4$.

Note that if $[v_1, v_2] = 0$, then $[(pv_1 + qv_2), (rv_1 + sv_2)] = 0$ 
for all $p,q,r,s \in \mathbb{Z}_3$. Hence, each context $C \in \mathcal{C}$ 
can be generated by $2$ commuting operators:
\begin{align*}
  \nonumber  && C  = \langle W(v_1), W(v_2) \rangle \\ 
   \nonumber &&= \{W(pv_1 + qv_2) \mid \forall p,q\in \mathbb{Z}_3 \}.
\end{align*}
where $[v_1, v_2] = 0$ and $v_2 \neq zv_1$ for any $z \in \mathbb{Z}_3$ 
(i.e, $v_1$ and $v_2$ are linearly independent). Each context contains 
$3^2 = 9$ operators (one is the identity operator). There are 40 such 
contexts for the 2-qutrit Heisenberg-Weyl group. The generators of the 
contexts are given by the symplectic vectors in `src/utils/contexts.py`.

In [54]:
# Printing the source code of the contexts module
import utils.contexts as contexts
Markdown(f"```python\n{inspect.getsource(contexts)}\n```")
# 

```python
"""Contexts for two qutrit Heisenberg-Weyl operators 
    -defined in terms of their symplectic vectors"""
import numpy as np

# Each context is specified by two symplectic vectors A and B.
# C[i,0] contains the A vector for context i
# C[i,1] contains the B vector for context i

C = np.array([
    # Context 1
    [[1,0,0,0], [0,0,1,0]],
    # Context 2
    [[1,0,0,0], [0,0,0,1]],
    # Context 3
    [[1,0,0,0], [0,0,1,1]],
    # Context 4
    [[1,0,0,0], [0,0,1,2]],
    # Context 5
    [[0,1,0,0], [0,0,1,0]],
    # Context 6
    [[0,1,0,0], [0,0,0,1]],
    # Context 7
    [[0,1,0,0], [0,0,1,1]],
    # Context 8
    [[0,1,0,0], [0,0,1,2]],
    # Context 9
    [[1,1,0,0], [0,0,1,0]],
    # Context 10
    [[1,1,0,0], [0,0,0,1]],
    # Context 11
    [[1,1,0,0], [0,0,1,1]],
    # Context 12
    [[1,1,0,0], [0,0,1,2]],
    # Context 13
    [[1,2,0,0], [0,0,1,0]],
    # Context 14
    [[1,2,0,0], [0,0,0,1]],
    # Context 15
    [[1,2,0,0], [0,0,1,1]],
    # Context 16
    [[1,2,0,0], [0,0,1,2]],
    # Context 17
    [[1,0,0,1], [0,1,1,0]],
    # Context 18
    [[1,0,0,1], [0,1,1,1]],
    # Context 19
    [[1,0,0,1], [0,1,1,2]],
    # Context 20
    [[0,1,1,0], [1,0,1,1]],
    # Context 21
    [[0,1,1,0], [1,0,2,1]],
    # Context 22
    [[0,1,1,1], [1,1,0,1]],
    # Context 23
    [[0,1,1,1], [1,1,2,0]],
    # Context 24
    [[0,1,1,2], [1,2,1,0]],
    # Context 25
    [[0,1,1,2], [1,2,0,1]],
    # Context 26
    [[1,0,1,1], [1,1,1,0]],
    # Context 27
    [[1,0,1,1], [1,1,0,2]],
    # Context 28
    [[1,0,2,1], [1,2,2,0]],
    # Context 29
    [[1,0,2,1], [1,2,0,2]],
    # Context 30
    [[1,1,2,0], [1,0,0,2]],
    # Context 31
    [[1,1,2,0], [1,0,2,2]],
    # Context 32
    [[1,2,1,0], [1,0,0,2]],
    # Context 33
    [[1,2,1,0], [1,0,1,2]],
    # Context 34
    [[1,1,0,2], [0,1,2,0]],
    # Context 35
    [[1,1,0,2], [0,1,2,2]],
    # Context 36
    [[1,2,0,2], [0,1,2,0]],
    # Context 37
    [[1,2,0,2], [0,1,2,1]],
    # Context 38
    [[1,1,1,2], [1,2,1,1]],
    # Context 39
    [[1,2,2,2], [1,1,2,1]],
    # Context 40
    [[0,1,2,1], [1,0,0,2]],
])

# For backward compatibility, provide A and B as views into C
A = C[:, 0]  # A[i] = C[i, 0]
B = C[:, 1]  # B[i] = C[i, 1]

```

With the pairs of orthogonal vectors defined in `src/utils/contexts.py`, any 
context can be generated of the form: 
\begin{equation*}
  C = \langle A(c), B(c) \rangle = \{W(pA(c) + qB(c)) \mid \forall p,q\in 
  \mathbb{Z}_3 \}
\end{equation*}

`src/utils.commutators.check_context_commutators` can be used to verify 
that for all contexts, the operators in the context pairwise commute.

## Empirical model
To compute the contextual fraction, we need the measurement statistics of 
joint measurements of each context. For a given quantum state $\rho$, the 
empirical model is the measurement statistics of joint outcomes indexed by 
the contexts. For example, for the CHSH scenario, a possible empirical 
model is
\begin{equation*}
     \begin{array}{c|cccc} 
         & (0,0) & (1,0) & (0,1) & (1,1) \\
         \hline(A, B) & 1 / 2 & 0 & 0 & 1 / 2 \\
         \left(A, B^{\prime}\right) & 3 / 8 & 1 / 8 & 1 / 8 & 3 / 8 \\
         \left(A^{\prime}, B\right) & 3 / 8 & 1 / 8 & 1 / 8 & 3 / 8 \\
         \left(A^{\prime}, B^{\prime}\right) & 1 / 8 & 3 / 8 & 3 / 8 & 1 / 8
     \end{array}
\end{equation*}
Each row corresponds to a context, and each column corresponds to an event - 
a joint outcome of measurement of all the operators in the context.

For 2-qutrit Heisenberg-Weyl operators, the outcome of a measurement is of 
the form $\omega^a$ where $\omega = e^{2\pi i / 3}$ and $a \in \mathbb{Z}_3$. 
We label the outcome $\omega^a$ as $a$. 

Note that while there are 8 operators in each context (ignoring the identity 
operator), there are **not** $3^8 = 6561$ possible outcomes. 
The operators of a context share a common eigenbasis. 

>For 2-qutrits, there are 9 common eigenvectors, and hence there are only 
>9 possible outcomes for each context. 
>This can also be seen from the fact that each context is generated by 
>2 operators, and each operator has 3 possible outcomes. 
>The outcomes of the two generators completely determine the outcomes of all 
>the other operators in the context.
>Hence, we can label the outcomes of a context by $(a,b)$ where $a,b \in 
>\mathbb{Z}_3$ are the outcomes of the two generators of the context. 

For the joint outcome $(a,b)$, the outcome of any operator $W(pA + qB)$ in 
the context is $\omega^{ap + bq}$.

To compute the probabilities of the joint outcomes, we need to construct 
projectors onto the common eigenvectors of the operators for each context.
This can be done as follows. 

By spectral decomposition, we can write any Heisenberg-Weyl $W$ as 
\begin{equation*}
  W = \sum_{a \in \mathbb{Z}_3} \omega^a \Pi_W^a
\end{equation*}
where $\Pi_W^a$ is the projector onto the eigenspace of $W$ with eigenvalue 
$\omega^a$. 

From the above, we get the form of the projector $\Pi_W^a$ as
\begin{equation*}
  \Pi_W^a = \frac{1}{3} \sum_{j\in \mathbb{Z}_3} \omega^{-aj} W^j
\end{equation*}

With this, we can construct the projectors for the joint outcomes for a 
context:
\begin{equation*}
  \Pi_C^r = \frac{1}{|C|} \sum_{W \in C} \omega^{-r_W} W
\end{equation*}
where $r_W$ is the outcome of the operator $W$ for the joint outcome $r$.

>Hence, for context $C = \langle A(c), B(c) \rangle$, the projector onto 
>the joint outcome $(a,b)$ is given by
>\begin{equation*}
>  \Pi_C^{(a,b)} = \frac{1}{9} \sum_{p,q \in \mathbb{Z}_3} \omega^{-ap - bq} 
>  W(pA(c) + qB(c))
>\end{equation*}

The projector for the context $c$ and the joint outcome $(a,b)$ can be 
computed using the function `src/utils.measurements.projector`:

In [55]:
# Printing the source code of the projector function
from utils.measurements import projector
Markdown(f"```python\n{inspect.getsource(projector)}\n```")

```python
def projector(c, a, b):
    """
    Calculate the measurement projector for a given context and output (a, b).

    For each pair (p, q), it performs the following steps:
      - Computes an exponent as (p * a + q * b) modulo 3.
      - Constructs an operator using a combination of A[c] and B[c] via the 
        pauli function.
      - Applies a phase factor by raising a global constant w to the power 
        of the negative exponent.
      - Accumulates the resulting operator.

    Parameters:
        c: Identifier/index selecting the context, used to index into A and B.
        a: Outcome of the first operator A in the context (0, 1, or 2).
        b: Outcome of the second operator B in the context (0, 1, or 2).

    Returns:
        The normalized measurement projector.
    """
    P = 0  # Initialize projector
    for p in range(3):
        for q in range(3):
            # if p == 0 and q == 0:
            #   continue
            exponent = (p * a + q * b) % 3  # Exponent for phase factor
            # Operator for (p, q) in context c
            op = operators.pauli(p * A[c] + q * B[c])
            term = w ** (-exponent) * op
            P += term
    return P / 9  

```

With the projectors defined, we can compute the empirical model for a 
quantum state $\rho$. We generate the empirical model in a vectorized form, 
i.e., the empirical model is a vector of probabilities where the first $9$ 
entries correspond to the probabilities of the joint outcomes of the first 
context, the next $9$ entries correspond to the probabilities of the joint 
outcomes of the second context, and so on. 

For the outcome $(a,b)$, we can interpret $(a,b)$ as the ternary 
representation of the index of the outcome.

Hence, within a given context $c$, the outcome $(a,b)$ corresponds to the 
entry at index $[9c + (3 a + b)]$ in the empirical model vector. For 40 
contexts, the empirical model is a vector of length $9 \times 40 = 360$. 

The empirical model for a given state can be computed using the function 
`src/utils.measurements.empirical_model`:

In [60]:
# Printing the source code of the empirical_model function
from utils.measurements import empirical_model
Markdown(f"```python\n{inspect.getsource(empirical_model)}\n```")

```python
def empirical_model(rho):
    """
    Calculate the vectorized empirical model for the given quantum state.

    This function computes the probabilities of joint measurement outcomes 
    for each context.
   
    For a given context c:
      - Generates the projectors for all pairs (a, b) in that context.
      - Computes the probabilities using the Born rule: 
        P(a, b) = Tr(rho @ P(a, b)).

    The computed probabilities are stored in a flattened vector of size 360 
    (each context contributes 9 entries).

    If the total probability for any context exceeds 1, a warning message 
    is printed.

    Parameters:
        rho (np.ndarray): The density matrix representing the quantum state. 

    Returns:
        np.ndarray: A 1D numpy array of length 360, where each segment of 
            9 elements corresponds to a measurement context.
    """
    E = np.zeros(360)  # Initialize empirical model vector
    for c in range(40):  # Range over contexts
        for a in range(3):
            for b in range(3):
                # Projectors precomputed using the projector function 
                # outside the loop
                P = projectors[c][a][b]
                E[9*c + (3 * a + b)] = np.trace(rho @ P).real  # Born rule
        tol = 1e-4  # Tolerance for slight numerical deviations
        if np.sum(E[9*c:9*c+9]) > 1 + tol:
            print("Sum of entries for context", c, ":", 
                  np.sum(E[9*c:9*c+9]))
    return E

```

## (Non)contextual fraction
Let's say we have a global value assignment $g$ for every Heisenberg-Weyl 
operator. That is, for each operator $W$, we know the measurement outcome 
$r_W = g(W) \in \mathbb{Z}_3$ deterministically. 

In the empirical model generated by this global assignment $g$, the 
probability of the joint outcome $r$ for a context $C$ is given by
\begin{equation*}
  p_C^g(r) = \begin{cases}
    1 & \text{if } r_W = g(W) \text{ for all } W \in C \\
    0 & \text{otherwise}
  \end{cases}
\end{equation*}

That is, each context has the probability $1$ for the joint outcome that is 
consistent with the global assignment $g$ and probability $0$ for all other 
joint outcomes.

For example, in the CHSH scenario, a global assignment $g$ could be:
\begin{equation*}
  g(A) = 0, g(A') = 1, g(B) = 0, g(B') = 1
\end{equation*}
The empirical model generated by this global assignment is:
\begin{equation*}
     \begin{array}{c|cccc}
         & (0,0) & (0,1) & (1,0) & (1,1) \\
         \hline(A, B) & 1 & 0 & 0 & 0 \\
         \left(A, B^{\prime}\right) & 0 & 1 & 0 & 0 \\
         \left(A^{\prime}, B\right) & 0 & 0 & 1 & 0 \\
         \left(A^{\prime}, B^{\prime}\right) & 0 & 0 & 0 & 1
     \end{array}
\end{equation*}

> A convex combination of empirical models is also a valid empirical model. 
> An empirical model is said to be **noncontextual** if it can be expressed 
> as a convex combination of empirical models generated by global assignments.

Any empirical model e can be decomposed into the form
\begin{equation*}
  e = \lambda e_{NC} + (1 - \lambda) e'
\end{equation*}
where $e_{NC}$ is a noncontextual empirical model. Note that this 
decomposition is not unique. By doing this, we explained a fraction of the 
events in terms of probabilistic combinations of global assignments.

Consider the convex decomposition where the weight of the noncontextual 
empirical model is maximized:
\begin{equation*}
  e = \lambda_{max} e_{NC} + (1 - \lambda_{max}) e'
\end{equation*}

For a decomposition of the above form, the 'residual' empirical model $e'$ 
is (strongly) contextual.

> This $\lambda_{max}$ is called the **noncontextual fraction** $NCF(e)$ of 
> the empirical model $e$, and $1 - \lambda_{max}$ is called the 
> **contextual fraction** $CF(e)$.

The empirical model $e$ admits the decomposition
\begin{equation*}
  e = NCFe_{NC} + (1 - NCF)e_{SC}
\end{equation*}

> An empirical model is said to be **contextual** if its contextual fraction 
> is greater than $0$.

## Incidence Matrix
Let $v_g$ be the vectorized form of the empirical model of a global assignment $g$.

> The incidence matrix $M$ is a matrix where each column corresponds to an empirical
model $v_g$ for a global assignment $g$. 

That is, $M$ is of the form
\begin{equation*}
  M(9c + (3a + b), g) = \begin{cases}
    1 & \text{if } g(A(c)) = a \text{ and } g(B(c)) = b \\
    0 & \text{otherwise}
  \end{cases}
\end{equation*}
where $c$ is the context index, $(a,b)$ is the joint outcome for the
context, and $g$ is the global assignment index.

To define the incidence matrix $M$, we first need to enumerate all the 
possible global assignments. We use the following lemma:

> **Lemma**: For 2-qutrit Heisenberg-Weyl operators, any global assignment
that is consistent with a quantum state is of the form
> \begin{equation*}
>  g(W(v)) = \lambda_g \cdot v
> \end{equation*}
> for some $\lambda_g \in \mathbb{Z}_3^4$.

Hence, there are $3^4 = 81$ such global assignments.
The global assignment $\lambda_g$ corresponding to the index $g$ 
can be generated by converting $g$ to a 4-digit base-3 number using `src/utils/ternary.py`.

The incidence matrix is constructed in `src/optimization/incidence_matrix.py`:

In [57]:
# Printing the source code of the incidence_matrix module
import optimization.incidence_matrix as incidence_matrix
Markdown(f"```python\n{inspect.getsource(incidence_matrix)}\n```")

```python
"""Generating the incidence matrix for all possible global assignments"""

import numpy as np
from scipy.sparse import lil_matrix

from utils.contexts import A, B
from utils.ternary import to_ternary

rows, cols = 360, 81  # rows = 9*40, cols = 3**4 
# Use lil_matrix for efficient construction
M_sparse = lil_matrix((rows, cols), dtype=int)

M = []  # Initialize incidence matrix
for g in range(cols):  # For each global assignment
    lam = to_ternary(g)  # Convert column number to ternary representation
    for c in range(40):
        a = np.dot(A[c], lam) % 3  # Compute outcome for A[c]
        b = np.dot(B[c], lam) % 3  # Compute outcome for B[c]
        # Position of the joint outcome (a,b) in context c
        row_index = 9*c + (3 * a + b)
        M_sparse[row_index, g] = 1

M = M_sparse.tocsr()  # Convert to csr format for efficiency in calculations

```

## Linear program for contextual fraction

Recall that an empirical model $e$ is noncontextual if it can be expressed 
as a convex combination of empirical models generated by global assignments.

Let $v^e$ be the vectorized form of the empirical model $e$. 
If $e$ is noncontextual, it can be expressed as a convex combination of the 
columns of the incidence matrix $M$.
This can be stated in terms of the linear equation:
\begin{equation*}
  v^e = M \cdot d
\end{equation*}
where $d$ is a probability vector. 

If $e$ is not noncontextual, then consider the decomposition
\begin{equation*}
  v^e = \lambda(M \cdot d) + (1 - \lambda) v'
\end{equation*}
where we have written the noncontextual part as a $M \cdot d$ for some $d$. 
We can absorb the factor $\lambda$ into the vector $d$ and write the above as
\begin{equation*}
  v^e = M \cdot b + (1 - \lambda) v'
\end{equation*}
where $b = \lambda d$ is a sub-probability distribution. Note that the sum 
of the entries of $b$ is $1\cdot b = \lambda$.

The noncontextual fraction $NCF(e)$ is the maximum value of $\lambda$ such 
that the above equation holds. We can write this as a linear program:
\begin{align*}
  \text{Find } & \quad b \in \mathbb{R}^{81} \\
  \text{maximizing } & \quad 1 \cdot b \\
  \text{subject to } & \quad M \cdot b \leq v^e \\
  \text{and } & \quad b \geq 0
\end{align*}

'src/optimization/lin_prog.py' contains the function `contextual_fraction` 
which implements this linear program using the `scipy.optimize.linprog` 
function.

In [58]:
# Printing the source code of the lin_prog module
import optimization.lin_prog as lin_prog
Markdown(f"```python\n{inspect.getsource(lin_prog)}\n```")

```python
import numpy as np
from scipy.optimize import linprog
from optimization.incidence_matrix import M
from utils.measurements import empirical_model


def contextual_fraction(rho):
    """
    Solve the linear program to maximize the contextual fraction.
    
    Args:
        rho: The density matrix/quantum state
        
    Returns:
        dict: Contains the optimization result with keys:
            - 'success': bool, whether optimization succeeded
            - 'b': array, optimal solution vector (if successful)
            - 'result': scipy.optimize.OptimizeResult object
    """
    # === Linear Program ===
    # maximize 1.b  -> minimize -1.b
    
    c = -np.ones(M.shape[1])  # Objective vector: length 81
    bounds = [(0, 1)] * M.shape[1]  # b >= 0
    
    # Empirical data
    E = empirical_model(rho)
    
    # Solve using HiGHS
    result = linprog(c, A_ub=M, b_ub=E, bounds=bounds, method='highs')
    
    # === Output ===
    output = {
        'success': result.success,
        'result': result
    }
    
    if result.success:
        output['b'] = 1 - np.dot(-c, result.x)
    
    return output

```

## Example usage
In `src/utils/states.py`, we have defined some example quantum states for 
which we can compute the contextual fraction.
For these states, we get the following contextual fractions:

In [59]:
# Create some test states and compute their contextual fractions
from utils.states import get_default_test_states
import optimization.lin_prog as lin_prog

# Get test states
states_dict = get_default_test_states()

# Compute contextual fractions for each state
print("Contextual fractions for different quantum states:")
print("=" * 50)
for name, rho in states_dict.items():
    result = lin_prog.contextual_fraction(rho)
    if result['success']:
        cf = result['b']
        print(f"{name}: {cf:.6f}")
    else:
        print(f"{name}: Optimization failed")

Contextual fractions for different quantum states:
Maximally Mixed State: -0.000000
Product State |00⟩: 0.000000
Maximally Entangled State: 0.000000
Custom Superposition: 0.000000
